# Big Data Exam Report @ UniBo a.y. 2023/2024

- Manuel Andruccioli
- Kelvin Olaiya

## Data Definition

In [ ]:
def getCharIndexes(line: String, char: Char): Seq[Int] = line.zipWithIndex.filter(_._1 == char).map(_._2) 
def splitAt(s: String, indices: Seq[Int]): Seq[String] = indices match {
    case h +: t => s.splitAt(h) match {
        case (a, b) => a +: splitAt(b, t.map(_ - h))
    }
    case Nil => Seq(s)
}
def parseCSVLine(l: String): Seq[String] = {
    val apices = getCharIndexes(l, '"').grouped(2).map { case Seq(a, b) => (a, b) }.toSeq
    val commas = getCharIndexes(l, ',').filter(i => !apices.exists { case (a, b) => a < i && i < b })
    return splitAt(l, commas).map(_.dropWhile(s => s == ',' || s == ' ')).filter(_ != "").map(_.replaceAll("^\"|\"$", ""))
}